### 1. Make sure to have completed 'Prerequisites' from README.md

### 2. Copy midi files to `data/{type}/{type_short}/` folder.
> Examples: `data/midi_data_x/x/`, `data/midi_data_y/y/`, `data/midi_data_xy/xy/`, `data/midi_data_y_neg/y_neg/`

### 3. Prepare name list

```
find data/midi_data_x/x -type f -name *.mid -o -name *.xml | cut -c 20- > data/midi_data_x/original-names.txt
```

The `cut` command uses 20- here but it depends on the path (`data/midi_data_x/x`) length.

### 4. Convert the data to json

Make sure the path `data/midi_data_x/processed/json/` exists and run:

```
python mmt/convert_lmd_full.py -n data/midi_data_x/original-names.txt -i data/midi_data_x/x/ -o data/midi_data_x/processed/json/
```

### 5. Extract notes

Make sure the path `data/midi_data_x/processed/notes/` exists and run:

```
python mmt/extract.py -d midi_data_x
```

### 6. Split training/validation/test sets

```
python mmt/split.py -d midi_data_x -v 0 -t 1
```

0 and 1 are the validation and test set ratios, respectively. All files are being assigned to the test set.

### 7. Download pretrained model

Download from https://drive.google.com/drive/folders/1HoKfghXOmiqi028oc_Wv0m2IlLdcJglQ?usp=share_link using gdown

```
gdown --id 1HoKfghXOmiqi028oc_Wv0m2IlLdcJglQ --folder
```

Copy the `sod-ape` model (best_model.pt) to `exp/midi_data_x/ape/checkpoints/`

### 8. Download pre-processed sod dataset

Download pre-processed sod dataset (sod_json.zip and sod_notes.zip) from https://drive.google.com/drive/folders/1owWu-Ne8wDoBYCFiF9z11fruJo62m_uK?usp=share_link using gdown

```
gdown --id 1owWu-Ne8wDoBYCFiF9z11fruJo62m_uK --folder
```

Extract the files (sod_json.zip and sod_notes.zip) to data/sod/processed/json and data/sod/processed/notes


### 9. Start training the sod-ape model to generate the train-args.json file

Make sure the folder `exp/sod/ape/checkpoints` exists and then run:

```
python mmt/train.py -d sod -o exp/sod/ape -g 0
```

This generates the `train-args.json` file in `exp/sod/ape/`. Copy this file to `exp/midi_data_x/ape/`.

### 10. Generate samples using the pre-trained model

```
python mmt/generate.py -d midi_data_x -o exp/midi_data_x/ape -g 0 -ns 4
```

### 11. Compute H

Using the files in `exp/midi_data_x/ape/samples/logits/` and `exp/midi_data_x/ape/samples/npy/`

In [1]:
import numpy as np
import scipy.stats as stats
import os

In [2]:
tuple_encoding = {"type" : 0,
                  "beat" : 1, 
                  "position": 2,
                  "pitch" : 3,
                  "duration" : 4,
                  "instrument": 5}

In [4]:
def read_logits(folder):
    """
    The logits folder (eg. /exp/midi_data_x/ape/samples/logits/) will have files like:
    0_16-beat-continuation-beat_logits.npy
    0_16-beat-continuation-duration_logits.npy
    0_16-beat-continuation-instrument_logits.npy
    0_16-beat-continuation-pitch_logits.npy
    0_16-beat-continuation-position_logits.npy
    0_16-beat-continuation-type_logits.npy
    1_16-beat-continuation-beat_logits.npy
    1_16-beat-continuation-duration_logits.npy
    1_16-beat-continuation-instrument_logits.npy
    1_16-beat-continuation-pitch_logits.npy
    1_16-beat-continuation-position_logits.npy
    1_16-beat-continuation-type_logits.npy
    .
    .
    .
    n_16-beat-continuation-beat_logits.npy
    n_16-beat-continuation-duration_logits.npy
    n_16-beat-continuation-instrument_logits.npy
    n_16-beat-continuation-pitch_logits.npy
    n_16-beat-continuation-position_logits.npy
    n_16-beat-continuation-type_logits.npy

    This function will read the logits and return a dictionary of the form:
    {0 : 
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()},
    1 :
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()},
    .
    .
    .
    n : 
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()}
    }
    """
    filenames = os.listdir(folder)
    sample_ids = [int(fname.split('_')[0]) for fname in filenames]
    logits = {id: dict() for id in sample_ids}

    for fname in filenames:
        if fname.endswith(".npy"):
            id = int(fname.split('_')[0])
            l_type = fname.split('-')[-1].split('_')[0]
            logits[id][l_type] = np.load(os.path.join(folder, fname)).squeeze()
    
    return logits

def read_truth(folder):
    """
    The npy folder (eg. /exp/midi_data_x/ape/samples/npy/) will have files like:
    0_truth.npy
    1_truth.npy
    .
    .
    .
    n_truth.npy 

    in addition to the generated files.

    This function will read the truth files and return a dictionary of the form:
    {0 : np.array(),
    1 : np.array(),
    .
    .
    .
    n : np.array()}
    """
    filenames = os.listdir(folder)
    
    truths = dict()
    for fname in filenames:
        if fname.endswith("_truth.npy"):
            id = int(fname.split('_')[0])
            truths[id] = np.load(os.path.join(folder, fname))

    return truths

def calc_entropy(logits, truth):
    """
    Given logits[i] of the form:

    {"type" : np.array(),
    "beat" : np.array(),
    "position" : np.array(),
    "pitch" : np.array(),
    "duration" : np.array(),
    "instrument" : np.array()}

    and truth[i] of the form: np.array(),

    this function will compute the entropy each of the 6 positions in the tuple representation:

    { "type" : entropy,
      "beat" : entropy,
      "position" : entropy,
      "pitch" : entropy,
      "duration" : entropy,
      "instrument" : entropy}
    """

    # select notes from truth after the first 16 beats
    selected_truth = truth[truth[:, 1] >= 16]

    # select rows from logits corresponding to the selected notes
    selected_logits = {l_type : logits[l_type][:len(selected_truth)] 
                       for l_type in logits.keys()}

    # convert logits to probabilities
    probs = {l_type : 1 / (1 + np.exp(-selected_logits[l_type])) for l_type in selected_logits.keys()}

    # select the probability of the truth note from probs
    selected_probs = {l_type : probs[l_type][np.arange(len(selected_truth)), 
                                                       selected_truth[:, tuple_encoding[l_type]]]
                        for l_type in probs.keys()}

    # compute entropy
    entropies = {l_type : stats.entropy(selected_probs[l_type], base=2) for l_type in selected_probs.keys()}

    return entropies
    

In [5]:
logits = read_logits('../exp/midi_data_x/ape/samples/logits/')

In [6]:
truths = read_truth('../exp/midi_data_x/ape/samples/npy/')

In [8]:
for id in logits.keys():
    print(id)
    print(calc_entropy(logits[id], truths[id]))
    print()

0
{'duration': 6.848968982830749, 'pitch': 7.186772490141254, 'position': 6.567864715856696, 'type': 7.209452874206438, 'beat': 6.812764571032318, 'instrument': 7.209453562137042}

1
{'beat': 5.906540516424541, 'position': 7.868171880443323, 'type': 7.8703636273459185, 'instrument': 7.8703636273459185, 'pitch': 7.831742515337562, 'duration': 7.866460309101936}

2
{'duration': 6.320386496779339, 'pitch': 6.518399753746138, 'type': 6.523561984994436, 'instrument': 6.523561297063833, 'position': 5.484863478045357, 'beat': 6.2521788653777515}

3
{'instrument': 6.67242535206706, 'duration': 6.570473348704983, 'pitch': 6.644028264687195, 'beat': 4.247927448752752, 'position': 5.998470058824936, 'type': 6.672426727928266}

